In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import yfinance as yf
import time
from datetime import datetime
from tqdm import tqdm
import os

# 설정 값
DELAY_BETWEEN_REQUESTS = 1.5  # 각 요청 사이 지연 시간 (초)
RETRY_WAIT_SECONDS = 5        # 실패 시 재시도 전 대기 시간 (초)
MAX_RETRIES = 3               # 최대 재시도 횟수

# 저장 경로
SAVE_DIR = "/content/drive/MyDrive/Data/FNC/Historical_Data"
os.makedirs(SAVE_DIR, exist_ok=True)

# 티커 파일 로딩
df_tickers = pd.read_csv("/content/drive/MyDrive/Data/FNC/Tickers4CL.csv")
tickers = df_tickers["[Symbol]"].dropna().unique()

# 각 티커별로 데이터 수집 및 저장
for ticker in tqdm(tickers, desc="Downloading historical data"):
    success = False
    for attempt in range(MAX_RETRIES):
        try:
            ticker_obj = yf.Ticker(ticker)
            data = ticker_obj.history(period="max", auto_adjust=False)

            if not data.empty:
                data = data.reset_index()
                data["Symbol"] = ticker

                # 필요한 컬럼 순서 지정
                if "Adj Close" not in data.columns:
                    print(f"[{ticker}] ❗ Adj Close 누락됨 — 확인 필요")
                else:
                    cols = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "Symbol"]
                    data = data[cols]

                filename = f"{ticker}.csv".replace("/", "-")  # 파일명에 슬래시 등 제거
                filepath = os.path.join(SAVE_DIR, filename)
                data.to_csv(filepath, index=False, encoding="utf-8-sig")
            success = True
            break
        except Exception as e:
            print(f"[{ticker}] 다운로드 실패 (시도 {attempt+1}/{MAX_RETRIES}) - {e}")
            time.sleep(RETRY_WAIT_SECONDS)
    time.sleep(DELAY_BETWEEN_REQUESTS)

print(f"✅ 개별 CSV 파일로 저장 완료 (경로: {SAVE_DIR})")


✅ 개별 CSV 파일로 저장 완료 (경로: /content/drive/MyDrive/Data/FNC/Historical_Data)
